In [1]:
pip install konlpy

     |████████████████████████████████| 19.4MB 5.1MB/s 
     |████████████████████████████████| 460kB 39.8MB/s 
     |████████████████████████████████| 92kB 9.9MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [37]:
#영화 리뷰 데이터 가져오기
import pandas as pd
lala_review = pd.read_csv('naver_preprocess.csv',encoding='utf-8') #score, text, idx, user, written_at, agree, disagree, body(text->특수문자, 숫자, 중복 제거 및 맞춤법, 띄어쓰기 교정->body)
lala_review = lala_review.dropna()
lala_review.head()

,score,text,idx,user,written_at,agree,disagree,body
0,10,극장에서 3번이나 봤습니다 그래도 보고싶은 너무나 행복한 영화입니다,12180460,후니뷘(beau****),2017.01.15 17:00,1,1,극장에서 3번이나 봤습니다 그래도 보고 싶은 너무나 행복한 영화입니다
1,10,꿈의나라로~!!!!,12180563,더리버리(rapp****),2017.01.15 17:16,2,1,꿈의 나라로
2,9,반전과 공감이 있는 로맨스 영화,12180612,조윤경(jykj****),2017.01.15 17:26,2,1,반전과 공감이 있는 로맨스 영화
3,8,마지막에 남녀가 만나서 사랑을 이뤘으면 행복한 결말이였지만 그렇지 못해서 더 애잔함...,12180615,상하(shtb****),2017.01.15 17:27,2,0,마지막에 남녀가 만나서 사랑을 이뤘으면 행복한 결말이었지만 그렇지 못해서 더 애잔함...
4,8,영상미는 진짜 깜짝 놀랄정도였다근데 난 개인적으로 여주가 극적으로 성공하는게 너무 ...,12180617,이소의(thdm****),2017.01.15 17:27,2,0,영상미는 진짜 깜짝 놀랄 정도였다 근데 난 개인적으로 여주가 극적으로 성공하는 게 ...


In [38]:
#라벨 달기
lala_review['sentiment'] = [0 if score<=5 else 1 for score in lala_review['score']] #0 for negative, 1 for positive
lala_review.head(5)

,score,text,idx,user,written_at,agree,disagree,body,sentiment
0,10,극장에서 3번이나 봤습니다 그래도 보고싶은 너무나 행복한 영화입니다,12180460,후니뷘(beau****),2017.01.15 17:00,1,1,극장에서 3번이나 봤습니다 그래도 보고 싶은 너무나 행복한 영화입니다,1
1,10,꿈의나라로~!!!!,12180563,더리버리(rapp****),2017.01.15 17:16,2,1,꿈의 나라로,1
2,9,반전과 공감이 있는 로맨스 영화,12180612,조윤경(jykj****),2017.01.15 17:26,2,1,반전과 공감이 있는 로맨스 영화,1
3,8,마지막에 남녀가 만나서 사랑을 이뤘으면 행복한 결말이였지만 그렇지 못해서 더 애잔함...,12180615,상하(shtb****),2017.01.15 17:27,2,0,마지막에 남녀가 만나서 사랑을 이뤘으면 행복한 결말이었지만 그렇지 못해서 더 애잔함...,1
4,8,영상미는 진짜 깜짝 놀랄정도였다근데 난 개인적으로 여주가 극적으로 성공하는게 너무 ...,12180617,이소의(thdm****),2017.01.15 17:27,2,0,영상미는 진짜 깜짝 놀랄 정도였다 근데 난 개인적으로 여주가 극적으로 성공하는 게 ...,1


In [39]:
lala_review_sentiment = lala_review.apply(lambda x: True if x[8]==1 else False, axis=1)
neg_num = len(lala_review_sentiment[lala_review_sentiment==False].index)
pos_num = len(lala_review_sentiment[lala_review_sentiment==True].index)
print("# of negative review: ", neg_num)
print("# of positive review: ", pos_num)

# of negative review:  77
# of positive review:  914


In [40]:
#감성사전 데이터 가져오기
polarity_df = pd.read_csv('polarity.csv')
polarity_df.head(10)

,ngram,freq,COMP,NEG,NEUT,None,POS,max.value,max.prop
0,가*/JKS,1,0.0,0.000000,0.000000,0.0,1.000000,POS,1.000000
1,가*/JKS;있/VV,1,0.0,0.000000,0.000000,0.0,1.000000,POS,1.000000
2,가*/JKS;있/VV;었/EP,1,0.0,0.000000,0.000000,0.0,1.000000,POS,1.000000
3,가*/VV,3,0.0,0.000000,0.000000,0.0,1.000000,POS,1.000000
4,가*/VV;ㄴ다*/EF,1,0.0,0.000000,0.000000,0.0,1.000000,POS,1.000000
5,가/JKC,17,0.0,0.470588,0.235294,0.0,0.294118,NEG,0.470588
6,가/JKC;되/VV,11,0.0,0.363636,0.272727,0.0,0.363636,NEG,0.363636
7,가/JKC;되/VV;ㄴ/ETM,2,0.0,0.000000,1.000000,0.0,0.000000,NEUT,1.000000
8,가/JKC;되/VV;ㄹ/ETM,1,0.0,0.000000,1.000000,0.0,0.000000,NEUT,1.000000
9,가/JKC;되/VV;어/EC,2,0.0,1.000000,0.000000,0.0,0.000000,NEG,1.000000


In [41]:
#감성사전 데이터를 이용해 딕셔너리 만들기
polarity_dict = {}
for i in range(len(polarity_df)):
  key = str()

  ngram = polarity_df.iloc[i][0].split(';')
  for data in ngram:

    word, tag = data.split('/')
    if word[-1] == '*':
      word = word[:-1]
    
    key += word + '/' + tag 

  neg_score = polarity_df.iloc[i][3]
  pos_score = polarity_df.iloc[i][6]
 
  polarity_dict[key] = {'neg_score': neg_score, 'pos_score': pos_score}

In [42]:
polarity_dict

{'가/JKS': {'neg_score': 0.46428571399999996, 'pos_score': 0.41071428600000004},
 '가/JKS있/VV': {'neg_score': 0.272727273, 'pos_score': 0.545454545},
 '가/JKS있/VV었/EP': {'neg_score': 0.0, 'pos_score': 1.0},
 '가/VV': {'neg_score': 0.545454545, 'pos_score': 0.181818182},
 '가/VVㄴ다/EF': {'neg_score': 0.0, 'pos_score': 1.0},
 '가/JKC': {'neg_score': 0.470588235, 'pos_score': 0.294117647},
 '가/JKC되/VV': {'neg_score': 0.363636364, 'pos_score': 0.363636364},
 '가/JKC되/VVㄴ/ETM': {'neg_score': 0.0, 'pos_score': 0.0},
 '가/JKC되/VVㄹ/ETM': {'neg_score': 0.0, 'pos_score': 0.0},
 '가/JKC되/VV어/EC': {'neg_score': 1.0, 'pos_score': 0.0},
 '가/JKC되/VV어야지요/EF': {'neg_score': 0.0, 'pos_score': 1.0},
 '가/JKC아니/VCN': {'neg_score': 0.666666667, 'pos_score': 0.166666667},
 '가/JKC아니/VCNㄴ가/EC': {'neg_score': 1.0, 'pos_score': 0.0},
 '가/JKC아니/VCN면/EC': {'neg_score': 0.0, 'pos_score': 1.0},
 '가/JKC아니/VCNㅂ니다/EF': {'neg_score': 1.0, 'pos_score': 0.0},
 '가/JKS가능/NNG': {'neg_score': 0.0, 'pos_score': 1.0},
 '가/JKS가능/NNG하/XSA'

In [43]:
#n-gram 단어를 감성사전에서 찾아 점수 구하기
from konlpy.tag import Kkma 

def kosac_polarity(text):

  """
  kosac 감성사전이 n-gram으로 이루어져 있기 때문에 리뷰에 대한 각 1-gram, 2-gram, 3-gram을 구하여 사전에 검색한 후, 각 점수를 합산하는 방식을 사용하였다.
  """
  sentiment = 0.0
  tokens_count = 0

  kkma = Kkma()
  tagged_text = kkma.pos(text) 

  #1-gram
  for word, tag in tagged_text:
    unigram = word + '/' + tag

    score_data = polarity_dict.get(unigram, None)
    if score_data == None:
      continue

    else:
      sentiment += score_data['pos_score'] - score_data['neg_score']      
      tokens_count += 1
    
  #2-gram
  for i in range(len(tagged_text)-1):
    word1, tag1 = tagged_text[i]
    word2, tag2 = tagged_text[i+1]
    bigram = word1 + '/' + tag1 + word2 + '/' + tag2

    score_data = polarity_dict.get(bigram, None)
    if score_data == None:
      continue

    else:
      sentiment += score_data['pos_score'] - score_data['neg_score']      
      tokens_count += 1

  #3-gram
  for i in range(len(tagged_text)-2):
    word1, tag1 = tagged_text[i]
    word2, tag2 = tagged_text[i+1]
    word3, tag3 = tagged_text[i+2]
    trigram = word1 + '/' + tag1 + word2 + '/' + tag2 + word3 + '/' + tag3

    score_data = polarity_dict.get(trigram, None)
    if score_data == None:
      continue

    else:
      sentiment += score_data['pos_score'] - score_data['neg_score']      
      tokens_count += 1

  #judgement call ? Default to positive or negative
  if not tokens_count:
    return 1
  
  # sum greater than 0 => positive sentiment
  if sentiment >= 0:
    return 1 
 
  # negative sentiment
  else:  
    return 0 
  

In [44]:
for index, (text, sentiment) in enumerate(zip(lala_review.iloc[:5].body, lala_review.iloc[:5].sentiment)):
    print(f'Text {index} : {text.strip()}')
    print('Sentiment:', sentiment)
    print('Predicted Sentiment polarity:', kosac_polarity(text))
    print('-'*60)

Text 0 : 극장에서 3번이나 봤습니다 그래도 보고 싶은 너무나 행복한 영화입니다
Sentiment: 1
Predicted Sentiment polarity: 1
------------------------------------------------------------
Text 1 : 꿈의 나라로
Sentiment: 1
Predicted Sentiment polarity: 1
------------------------------------------------------------
Text 2 : 반전과 공감이 있는 로맨스 영화
Sentiment: 1
Predicted Sentiment polarity: 1
------------------------------------------------------------
Text 3 : 마지막에 남녀가 만나서 사랑을 이뤘으면 행복한 결말이었지만 그렇지 못해서 더 애잔함이 묻어나는 아름다운 영화
Sentiment: 1
Predicted Sentiment polarity: 1
------------------------------------------------------------
Text 4 : 영상미는 진짜 깜짝 놀랄 정도였다 근데 난 개인적으로 여주가 극적으로 성공하는 게 너무 억지스러웠음
Sentiment: 1
Predicted Sentiment polarity: 0
------------------------------------------------------------


In [50]:
#각 리뷰에 대한 감성 분석(분류)하기
from sklearn.metrics import classification_report

y_true = lala_review['sentiment'].tolist()
y_pred = []

for i in range(len(lala_review)):
  text = lala_review.iloc[i].body
  sentiment_pred = kosac_polarity(text)
  y_pred.append(sentiment_pred)

#감성 분석(분류) 결과 평가하기
target_names = ['negative', 'positive']
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.11      0.36      0.17        77
           1       0.93      0.75      0.83       914

    accuracy                           0.72       991
   macro avg       0.52      0.56      0.50       991
weighted avg       0.87      0.72      0.78       991

